# Hierarchical Clustering

## Autoreload

Autoreload allows the notebook to dynamically load code: if we update some helper functions *outside* of the notebook, we do not need to reload the notebook.

In [2]:
%load_ext autoreload
%autoreload 2

## Imports

In [3]:
# Cornerstones:
import numpy as np
import pandas as pd
# Fundamentals:
import os
import sys
# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
# Scraping? Again??
import procyclingstats as pcs


# Imports more related to the clustering itself


#from utility.data_understanding import *

Global variables, constants, etc.

In [4]:
sys.path.append('../dataset/')
sys.path.append('../utility/')

# Global variables, constants
IMAGE_DIR = '../images/'
RANDOM_STATE = 42   # we define a random state to make the results reproducible
RUN_SLOW_STUFF = True

## Dataframes

We've spent a lot of effort into engineering new features, understanding them, etc. Whether we've done this well and whether is enough it's a different story.

The point here is that - after the understanding, engineering and transformation - we have some cleaned-up versions of the datasets.

In [6]:
cyclist_df = pd.read_csv('../dataset/cyclists_cleaned.csv')
races_df = pd.read_csv('../dataset/races_cleaned.csv')

In [8]:
cyclist_df.columns

Index(['_url', 'name', 'birth_year', 'weight', 'height', 'nationality', 'bmi',
       'race_count', 'experience_level', 'total_points', 'victories_by_points',
       'average_position'],
      dtype='object')

In [9]:
races_df.columns

Index(['_url', 'name', 'stage', 'stage_type', 'points', 'uci_points', 'length',
       'climb_total', 'profile', 'startlist_quality', 'date', 'position',
       'cyclist', 'cyclist_age', 'is_tarmac', 'delta', 'time', 'steepness',
       'season', 'is_staged', 'race_country', 'norm_points',
       'age_performance_index', 'quality_adjusted_points', 'normalized_length',
       'normalized_quality', 'normalized_steepness', 'profile_adjustment',
       'stamina_index'],
      dtype='object')